# Background
- **Author**: `<郭伊軒>`
- **Created At**: `<2025-11-1>`
- **Path to Training Data： discount-timing-DE.csv**
- **Path to Testing Data： discount-timing-DE.csv**
- **Model Specification 
    - Method：logistic regression
    - Variables：  
    ['Age', 'PlayerGrowthRate1W', 'FollowersGrowthRate1W', 'PositiveRateGrowthRate1W', 'SalePeriod', 'AccumulatedPositiveRate', 'DLC_sum_1W', 'Sequel_sum_1W']
    - Tuning Parameters：
    - Optimization Method：
- **Main Findings and Takeaways：**
    - In-sample `<metric>`:
    - Out-sample `<metric>`:
- **Future Direciton：**

In [13]:
# Load packages here
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import statsmodels.api as sm


In [68]:
# Load the TRAINING data here and please finish all the data manipulation here.
input_data_file = "/Users/10610/Desktop/114-1 資料/steam-project/discount-timing-DE.csv"
df = pd.read_csv(input_data_file)

#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
# Finish this block by printing the first ten observations of the data.
# Note:
# - You may only read data from /data/processed.
# - Files in /data/processed should already be cleaned and prepared for analysis.
# - Beyond simple filtering of observations or generating a small number of variables,
#   further data manipulation is not allowed. If more extensive changes are needed,
#   update the source data instead.


In [69]:
df['DiscountDuringSale'] = np.where(
    (df['DiscountOrNot'] == 1) & (df['SalePeriod'] == 1), 
    1, 
    0
)

df['DiscountOutOfSale'] = np.where(
    (df['DiscountOrNot'] == 1) & (df['SalePeriod'] == 0), 
    1, 
    0
)
survival = [294100, 108600, 242760, 457140, 376210, 233860, 323190, 644930]
df['TypeSurvival'] = np.where(
    df['GameID'].isin(survival),
    1,
    0
)
openWorld = [381210, 1091500, 108600, 242760, 244850, 376210, 233860]
df['TypeOpenWorld'] = np.where(
    df['GameID'].isin(openWorld),
    1,
    0
)

df.describe().T

,count,mean,std,min,25%,50%,75%,max
GameID,23938.0,461376.742000,298559.181056,10.000000,244850.000000,431730.000000,644930.000000,1.145360e+06
MultiPlayer,23938.0,0.464241,0.498730,0.000000,0.000000,0.000000,1.000000,1.000000e+00
ConstantDiscount,23938.0,0.214387,0.410405,0.000000,0.000000,0.000000,0.000000,1.000000e+00
DiscountOrNot,23938.0,0.019885,0.139607,0.000000,0.000000,0.000000,0.000000,1.000000e+00
DiscountDuration,23938.0,0.221196,1.715483,0.000000,0.000000,0.000000,0.000000,3.200000e+01
Age,23938.0,7.634427,4.458471,2.389041,4.951370,6.323288,8.479452,2.484658e+01
AccumulatedPositiveRate,23938.0,0.928061,0.064186,0.738751,0.905517,0.953165,0.972651,9.929734e-01
SalePeriod,23938.0,0.146420,0.353534,0.000000,0.000000,0.000000,0.000000,1.000000e+00
DiscountDuringSale,23938.0,0.008647,0.092590,0.000000,0.000000,0.000000,0.000000,1.000000e+00
DiscountOutOfSale,23938.0,0.011237,0.105411,0.000000,0.000000,0.000000,0.000000,1.000000e+00


### The actual modeling starts below
For the remaining blocks, make sure you have followed the guidelines as specified in [專案資料夾結構、檔案命名與文件規範](https://docs.google.com/document/d/1sl6gEFMdmiGsiNjLe17UmZ30xKxq15U0Mb2B-Jvusxg/edit?tab=t.33iie8ybx7s4).


季節性1W

In [70]:
df_dummies = pd.get_dummies(df, columns=['GameID'], drop_first=True)

# 應變數與自變數
y = df_dummies['DiscountDuringSale']
X = df_dummies[['Age', 'PlayerGrowthRate1W', 'FollowersGrowthRate1W', 'PositiveRateGrowthRate1W', 
                'SalePeriod', 'AccumulatedPositiveRate', 
                'DLC_sum_1W', 'Sequel_sum_1W'] +
               [col for col in df_dummies.columns if col.startswith('GameID_')]]

for col in X.columns:
    if X[col].dtype == 'bool':
        X[col] = X[col].astype(int)

C:\Users\10610\AppData\Local\Temp\ipykernel_2952\2158198301.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X[col] = X[col].astype(int)
C:\Users\10610\AppData\Local\Temp\ipykernel_2952\2158198301.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X[col] = X[col].astype(int)
C:\Users\10610\AppData\Local\Temp\ipykernel_2952\2158198301.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveat

In [24]:
# Load the TESTING data here and please finish all the data manipulation here.
# Finish this block by printing the first ten observations of the data.
X.describe().T

,count,mean,std,min,25%,50%,75%,max
const,23938.0,1.000000,0.000000,1.000000,1.000000,1.000000,1.000000,1.000000
Age,23938.0,7.634427,4.458471,2.389041,4.951370,6.323288,8.479452,24.846575
PlayerGrowthRate1W,23938.0,0.020047,0.288771,-0.592919,-0.063356,-0.014085,0.035173,11.311195
FollowersGrowthRate1W,23938.0,0.001576,0.001482,-0.000137,0.000640,0.001111,0.002078,0.022968
PositiveRateGrowthRate1W,23938.0,0.000017,0.000332,-0.015190,-0.000025,0.000005,0.000044,0.006060
SalePeriod,23938.0,0.146420,0.353534,0.000000,0.000000,0.000000,0.000000,1.000000
DiscountDuration,23938.0,0.221196,1.715483,0.000000,0.000000,0.000000,0.000000,32.000000
AccumulatedPositiveRate,23938.0,0.928061,0.064186,0.738751,0.905517,0.953165,0.972651,0.992973
DLC_sum_1W,23938.0,0.004679,0.071822,0.000000,0.000000,0.000000,0.000000,2.000000
Sequel_sum_1W,23938.0,0.001170,0.034181,0.000000,0.000000,0.000000,0.000000,1.000000


In [71]:
# Include the model specification here.  Please describe the model by stating, including
# type of method used, the variables included, the hyperparameters, and
# the method of optimization. 
# 建立遊戲虛擬變數 (GameID fixed effects)


X = sm.add_constant(X)
# 邏輯迴歸
logit_model = sm.Logit(y, X).fit()
print(logit_model.summary())
 

         Current function value: 0.027483
         Iterations: 35
                           Logit Regression Results                           
Dep. Variable:     DiscountDuringSale   No. Observations:                23938
Model:                          Logit   Df Residuals:                    23902
Method:                           MLE   Df Model:                           35
Date:                Sat, 01 Nov 2025   Pseudo R-squ.:                  0.4469
Time:                        22:49:11   Log-Likelihood:                -657.90
converged:                      False   LL-Null:                       -1189.5
Covariance Type:            nonrobust   LLR p-value:                1.582e-200
                               coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------------------
const                      -38.7537   2.27e+04     -0.002      0.999   -4.45e+04    4.44e+04
Age                    

c:\Users\10610\anaconda3\Lib\site-packages\statsmodels\base\model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


非季節性1W

In [46]:
input_data_file = "/Users/10610/Desktop/114-1 資料/steam-project/discount-timing-DE.csv"
df = pd.read_csv(input_data_file)

df['DiscountDuringSale'] = np.where(
    (df['DiscountOrNot'] == 1) & (df['SalePeriod'] == 1), 
    1, 
    0
)

df['DiscountOutOfSale'] = np.where(
    (df['DiscountOrNot'] == 1) & (df['SalePeriod'] == 0), 
    1, 
    0
)
#df = df[df['SalePeriod'] == 0]
df_dummies = pd.get_dummies(df, columns=['GameID'], drop_first=True)

# 應變數與自變數
y = df_dummies['DiscountOutOfSale']
X = df_dummies[['Age', 'PlayerGrowthRate1W', 'FollowersGrowthRate1W', 'PositiveRateGrowthRate1W', 
                'AccumulatedPositiveRate', 
                'DLC_sum_1W', 'Sequel_sum_1W'] +
               [col for col in df_dummies.columns if col.startswith('GameID_')]]

for col in X.columns:
    if X[col].dtype == 'bool':
        X[col] = X[col].astype(int)

C:\Users\10610\AppData\Local\Temp\ipykernel_2952\807671980.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X[col] = X[col].astype(int)
C:\Users\10610\AppData\Local\Temp\ipykernel_2952\807671980.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X[col] = X[col].astype(int)
C:\Users\10610\AppData\Local\Temp\ipykernel_2952\807671980.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats i

In [47]:
X = sm.add_constant(X)
# 邏輯迴歸
logit_model = sm.Logit(y, X).fit()
print(logit_model.summary())

         Current function value: 0.058186
         Iterations: 35
                           Logit Regression Results                           
Dep. Variable:      DiscountOutOfSale   No. Observations:                23938
Model:                          Logit   Df Residuals:                    23903
Method:                           MLE   Df Model:                           34
Date:                Sat, 01 Nov 2025   Pseudo R-squ.:                 0.05562
Time:                        17:31:36   Log-Likelihood:                -1392.9
converged:                      False   LL-Null:                       -1474.9
Covariance Type:            nonrobust   LLR p-value:                 5.850e-19
                               coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------------------
const                       -3.6880     11.746     -0.314      0.754     -26.709      19.333
Age                    

c:\Users\10610\anaconda3\Lib\site-packages\statsmodels\base\model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


非季節性1M

In [53]:
# Report the performance metrics for training data in this block.
input_data_file = "/Users/10610/Desktop/114-1 資料/steam-project/discount-timing-DE.csv"
df = pd.read_csv(input_data_file)

df['DiscountDuringSale'] = np.where(
    (df['DiscountOrNot'] == 1) & (df['SalePeriod'] == 1), 
    1, 
    0
)

df['DiscountOutOfSale'] = np.where(
    (df['DiscountOrNot'] == 1) & (df['SalePeriod'] == 0), 
    1, 
    0
)
#df = df[df['SalePeriod'] == 0]
df_dummies = pd.get_dummies(df, columns=['GameID'], drop_first=True)

# 應變數與自變數
y = df_dummies['DiscountOutOfSale']
X = df_dummies[['Age', 'PlayerGrowthRate1M', 'FollowersGrowthRate1M', 'PositiveRateGrowthRate1M', 
                'AccumulatedPositiveRate', 
                'DLC_sum_1M', 'Sequel_sum_1M'] +
               [col for col in df_dummies.columns if col.startswith('GameID_')]]

for col in X.columns:
    if X[col].dtype == 'bool':
        X[col] = X[col].astype(int)
 

C:\Users\10610\AppData\Local\Temp\ipykernel_2952\3684924110.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X[col] = X[col].astype(int)
C:\Users\10610\AppData\Local\Temp\ipykernel_2952\3684924110.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X[col] = X[col].astype(int)
C:\Users\10610\AppData\Local\Temp\ipykernel_2952\3684924110.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveat

In [54]:
# Report the performance metrics for testing data in this block.
X = sm.add_constant(X)
# 邏輯迴歸
logit_model = sm.Logit(y, X).fit()
print(logit_model.summary())
 

         Current function value: 0.058137
         Iterations: 35
                           Logit Regression Results                           
Dep. Variable:      DiscountOutOfSale   No. Observations:                23938
Model:                          Logit   Df Residuals:                    23903
Method:                           MLE   Df Model:                           34
Date:                Sat, 01 Nov 2025   Pseudo R-squ.:                 0.05641
Time:                        17:53:25   Log-Likelihood:                -1391.7
converged:                      False   LL-Null:                       -1474.9
Covariance Type:            nonrobust   LLR p-value:                 2.280e-19
                               coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------------------
const                       -0.6511     11.921     -0.055      0.956     -24.017      22.715
Age                    

c:\Users\10610\anaconda3\Lib\site-packages\statsmodels\base\model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


增加遊戲類別

In [ ]:
# 應變數與自變數
y = df_dummies['DiscountOrNot']
X = df_dummies[['Age', "MultiPlayer", "TypeSurvival", 'TypeOpenWorld', 'PlayerGrowthRate1W', 'FollowersGrowthRate1W', 'PositiveRateGrowthRate1W', 
                'SalePeriod', 'AccumulatedPositiveRate', 
                'DLC_sum_1W', 'Sequel_sum_1W']]

for col in X.columns:
    if X[col].dtype == 'bool':
        X[col] = X[col].astype(int)

,count,mean,std,min,25%,50%,75%,max
Age,23938.0,7.634427,4.458471,2.389041,4.951370,6.323288,8.479452,24.846575
MultiPlayer,23938.0,0.464241,0.498730,0.000000,0.000000,0.000000,1.000000,1.000000
TypeSurvival,23938.0,0.285655,0.451735,0.000000,0.000000,0.000000,1.000000,1.000000
TypeOpenWorld,23938.0,0.250021,0.433034,0.000000,0.000000,0.000000,0.750000,1.000000
PlayerGrowthRate1W,23938.0,0.020047,0.288771,-0.592919,-0.063356,-0.014085,0.035173,11.311195
FollowersGrowthRate1W,23938.0,0.001576,0.001482,-0.000137,0.000640,0.001111,0.002078,0.022968
PositiveRateGrowthRate1W,23938.0,0.000017,0.000332,-0.015190,-0.000025,0.000005,0.000044,0.006060
SalePeriod,23938.0,0.146420,0.353534,0.000000,0.000000,0.000000,0.000000,1.000000
AccumulatedPositiveRate,23938.0,0.928061,0.064186,0.738751,0.905517,0.953165,0.972651,0.992973
DLC_sum_1W,23938.0,0.004679,0.071822,0.000000,0.000000,0.000000,0.000000,2.000000


In [77]:
# Report the performance metrics for testing data in this block.
X = sm.add_constant(X)
# 邏輯迴歸
logit_model = sm.Logit(y, X).fit()
print(logit_model.summary())

Optimization terminated successfully.
         Current function value: 0.089671
         Iterations 14
                           Logit Regression Results                           
Dep. Variable:          DiscountOrNot   No. Observations:                23938
Model:                          Logit   Df Residuals:                    23927
Method:                           MLE   Df Model:                           10
Date:                Sat, 01 Nov 2025   Pseudo R-squ.:                 0.08115
Time:                        22:53:14   Log-Likelihood:                -2146.5
converged:                       True   LL-Null:                       -2336.1
Covariance Type:            nonrobust   LLR p-value:                 2.583e-75
                               coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------------------
const                       -3.8550      0.771     -4.997      0.000      -5.36